In [1]:
import os
import cv2
import numpy as np
import json
import sys
import json
import datetime
import skimage.draw
import tensorflow as tf
import matplotlib.pyplot as plt
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn.visualize import display_instances
import warnings
warnings.filterwarnings('ignore')

C:\Users\owner\anaconda3\envs\MLINSIGHT\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\owner\anaconda3\envs\MLINSIGHT\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\owner\anaconda3\envs\MLINSIGHT\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\owner\anaconda3\envs\MLINSIGHT\lib\site-packages\ten

In [2]:
# Root directory of the project
ROOT_DIR = os.path.abspath("./")

# To find local version of the library
sys.path.append(ROOT_DIR)

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [3]:
# Define the Model Configuration
class BarChartConfig(Config):
    # Give the configuration a name
    NAME = "barchart"
    
    #GPU_COUNT = 1
    
    IMAGES_PER_GPU = 1
    
    # Number of classes plus background
    NUM_CLASSES = 1 + 7  # Background + (Title, Legend, X-label, Y-label, Bar, X-value, Y-value)
    
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10
    
    # Skip detections with < 80% confidence
    DETECTION_MIN_CONFIDENCE = 0.8

In [4]:
class BarChartDataset(utils.Dataset):
    def load_barchart(self, dataset_dir, subset):
        """
        Load bar chart dataset from JSON annotations.
        """
        # Define class names (same as in annotation)
        self.add_class("barchart", 1, "title")
        self.add_class("barchart", 2, "legend")
        self.add_class("barchart", 3, "x-label")
        self.add_class("barchart", 4, "y-label")
        self.add_class("barchart", 5, "bars")
        self.add_class("barchart", 6, "x-ticks")
        self.add_class("barchart", 7, "y-ticks")

        # Validate subset type
        assert subset in ["training", "validation"], "Subset must be 'training' or 'validation'"
        json_file = os.path.join(dataset_dir, subset, f"{subset}_data.json")

        # Load annotations
        with open(json_file) as f:
            annotations1 = json.load(f) 

        # Ensure the correct key is accessed
        if "_via_img_metadata" not in annotations1:
            raise KeyError("Error: '_via_img_metadata' not found in JSON file.")

        # Extract annotations
        annotations = annotations1["_via_img_metadata"]

        for key, annotation in annotations.items():  # VIA stores metadata as a dictionary
            if "filename" not in annotation:
                print(f"Skipping entry {key}: Missing 'filename' key.")
                continue  # Skip entries without filenames
        
        # Loop through each image annotation
        for image_id, a in annotations.items():
            image_path = os.path.join(dataset_dir, subset, a["filename"])
            height, width = self._get_image_size(image_path)

            # Add image and annotations to dataset
            self.add_image(
                source="barchart",
                image_id=image_id,
                path=image_path,
                width=width,
                height=height,
                annotations=a["regions"]
            )

    def _get_image_size(self, image_path):
        """Helper function to get image dimensions"""
        image = skimage.io.imread(image_path)
        return image.shape[:2]

    def load_mask(self, image_id):
        """
        Generate instance masks for an image.
        """
        info = self.image_info[image_id]
        annotations = info["annotations"]
        num_instances = len(annotations)

        # Create an empty mask array
        mask = np.zeros([info["height"], info["width"], num_instances], dtype=np.uint8)
        class_ids = []

        # Iterate through annotations
        for i, annotation in enumerate(annotations):
            x = annotation["shape_attributes"]["x"]
            y = annotation["shape_attributes"]["y"]
            w = annotation["shape_attributes"]["width"]
            h = annotation["shape_attributes"]["height"]

            # Draw filled rectangle for each annotation
            mask[y:y+h, x:x+w, i] = 1

            # Assign class ID
            class_name = annotation["region_attributes"]["class"]
            class_id = self.class_names.index(class_name) if class_name in self.class_names else -1
            class_ids.append(class_id)

        return mask.astype(np.bool_), np.array(class_ids, dtype=np.int32)

In [ ]:
import matplotlib.pyplot as plt
import os

def train_model(model):
    """
    Train the Mask R-CNN model on the Bar Chart dataset and track training/validation losses.
    """
    dataset_dir = "C:\\Users\\owner\\Downloads\\New project\\Mask_RCNN\\Dataset"

    # Load training dataset
    dataset_train = BarChartDataset()
    dataset_train.load_barchart(dataset_dir, "training")
    dataset_train.prepare()

    # Load validation dataset
    dataset_val = BarChartDataset()
    dataset_val.load_barchart(dataset_dir, "validation")
    dataset_val.prepare()

    print("Training network heads")

    # To track training and validation losses
    class MetricsHistory:
        def __init__(self):
            self.train_losses = []
            self.val_losses = []

        def on_epoch_end(self, epoch, logs):
            # Append metrics from the logs to track loss
            self.train_losses.append(logs["loss"])
            self.val_losses.append(logs["val_loss"])

    # Initialize metrics history tracker
    metrics_history = MetricsHistory()

    # Define the custom callback to record loss metrics
    class CustomCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            if logs:
                metrics_history.on_epoch_end(epoch, logs)

    # Train the model
    model.train(
        train_dataset=dataset_train,
        val_dataset=dataset_val,
        learning_rate=config.LEARNING_RATE,
        epochs=10,
        layers="heads",
        custom_callbacks=[CustomCallback()]  # Include custom metrics callback
    )

    # Plot training and validation loss
    epochs = range(1, len(metrics_history.train_losses) + 1)
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, metrics_history.train_losses, 'r', label="Training Loss")
    plt.plot(epochs, metrics_history.val_losses, 'b', label="Validation Loss")
    plt.title("Training and Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

# Create configuration and model
config = BarChartConfig()
MODEL_DIR = "C:\\Users\\owner\\Downloads\\New project\\Mask_RCNN\\logs"
COCO_MODEL_PATH = "C:\\Users\\owner\\Downloads\\New project\\Mask_RCNN\\mask_rcnn_coco.h5"

model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

# Ensure COCO pre-trained weights exist
if not os.path.exists(COCO_MODEL_PATH):
    print(f"Downloading COCO weights to {COCO_MODEL_PATH}...")
    utils.download_trained_weights(COCO_MODEL_PATH)

# Load COCO weights (excluding incompatible layers)
print("Loading COCO weights...")
model.load_weights(COCO_MODEL_PATH, by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                            "mrcnn_bbox", "mrcnn_mask"])

# Start training
train_model(model)







Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead
Loading COCO weights...
Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: C:\Users\owner\Downloads\New project\Mask_RCNN\logs\barchart20250205T0818\mask_rcnn_barchart_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         